In [8]:
import neat
import pandas as pd
import joblib

In [9]:
# 1. Загрузка наборов данных
iris = pd.read_csv('dataset_1.csv')
diabetes = pd.read_csv('dataset_12.csv')

In [10]:
# Функция для подготовки данных
def prepare_data(X, y):
    X = X.to_numpy()  # Преобразуем DataFrame в массив NumPy
    y = y.to_numpy()  # Преобразуем Series в массив NumPy
    data = list(zip(X, y))
    return data

In [11]:
# Конфигурация NEAT
def run_neat(config_file, dataset, name, generations=50):
    # Подготовка данных
    
    figure_dict = {
    'circle': 0,
    'rectangle': 1
    }
    color_dict = {
    'red' : 0, 
    'blue' : 1, 
    'green' : 2, 
    'yellow' : 3
    }
    dataset['figure1'] = dataset['figure1'].map(figure_dict)
    dataset['figure2'] = dataset['figure2'].map(figure_dict)
    try:
        dataset['color1'] = dataset['color1'].map(color_dict)
        dataset['color2'] = dataset['color2'].map(color_dict)
    except:
        print()
    X = dataset.drop('collision', axis=1)
    y = dataset['collision']
    inputs = X.shape[1]  # Количество входов = количество столбцов в X
    training_data = prepare_data(X, y)
    
    # Загрузка конфигурации
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        config_file)
    
    # Создание популяции
    pop = neat.Population(config)
    
    # Добавление репортеров
    pop.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    
    # Функция оценки
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)
            error = 0
            for xi, xo in training_data:
                output = net.activate(xi)
                error += (output[0] - xo) ** 2
            genome.fitness = -error  # Чем меньше ошибка, тем выше приспособленность
    
    # Запуск эволюции
    winner = pop.run(eval_genomes, generations)
    
    # Возвращаем лучшую сеть
    return winner, stats

In [12]:
# Создание конфигурационных файлов
def create_config(filename, num_inputs, num_outputs=1):
    with open(filename, 'w') as f:
        f.write(f"""[NEAT]
fitness_criterion     = max
fitness_threshold     = 1.0
pop_size              = 200
reset_on_extinction   = False

[DefaultGenome]
# Параметры узлов
num_inputs              = {num_inputs}
num_hidden              = 0
num_outputs             = {num_outputs}
feed_forward            = True
initial_connection      = full
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient = 0.5

# Параметры инициализации смещений
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# Параметры инициализации характеристик узлов (response)
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.5
response_mutate_rate    = 0.1
response_replace_rate   = 0.2

# Параметры активационных функций
activation_default      = sigmoid
activation_mutate_rate  = 0.05
activation_options      = sigmoid tanh relu

# Параметры мутаций
node_add_prob           = 0.1
node_delete_prob        = 0.2

# Параметры агрегационных функций
aggregation_default         = sum
aggregation_mutate_rate     = 0.1
aggregation_options         = sum
aggregation_init_mean       = 0.0
aggregation_init_stdev      = 1.0
aggregation_max_value       = 30.0
aggregation_min_value       = -30.0

# Параметры весовых коэффициентов
weight_init_mean         = 0.0
weight_init_stdev        = 1.0
weight_max_value         = 30.0
weight_min_value         = -30.0
weight_mutate_power      = 0.5
weight_mutate_rate       = 0.8
weight_replace_rate      = 0.1

# Параметры активации соединений
enabled_default         = True
enabled_mutate_rate     = 0.1
conn_add_prob           = 0.3
conn_delete_prob        = 0.5

[DefaultSpeciesSet]
compatibility_threshold = 2.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 50

[DefaultReproduction]
elitism            = 3
survival_threshold = 0.2
""")

In [13]:
# Создание конфигураций
create_config('config_1dataset', 8)
create_config('config_2dataset', 20)

In [14]:
print("Обучение модели для маленького датасета...")
data1_winner, data1_stats = run_neat('config_1dataset', iris, "1dataset")
print("\nОбучение модели для большого датасета...")
data2_winner, data2_stats = run_neat('config_2dataset', diabetes, "2dataset")

Обучение модели для маленького датасета...


 ****** Running generation 0 ****** 

Population's average fitness: -25.01042 stdev: 4.09724
Best fitness: -16.99998 - size: (1, 8) - species 1 - id 29
Average adjusted fitness: 0.460
Mean genetic distance 1.646, standard deviation 0.538
Population of 200 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   117    -17.0    0.465     0
     2    0    81    -20.0    0.454     0
     3    0     2       --       --     0
Total extinctions: 0
Generation time: 0.136 sec

 ****** Running generation 1 ****** 

Population's average fitness: -9318.27787 stdev: 101199.27815
Best fitness: -16.99998 - size: (1, 8) - species 1 - id 29
Average adjusted fitness: 0.995
Mean genetic distance 1.711, standard deviation 0.526
Population of 201 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    83    -17.0    0.999     1
     2 

In [19]:
# Сравнение моделей
print("Сравнение моделей:")
print(f"1 датасет - лучшая приспособленность: {data1_winner.fitness}")
print(f"2 датасет - лучшая приспособленность: {data2_winner.fitness}")

Сравнение моделей:
1 датасет - лучшая приспособленность: -8.990854798037256
2 датасет - лучшая приспособленность: -332.56477662906934


In [16]:
# Создание сетей для подсчета узлов и связей
config_smalldata = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        'config_1dataset')
config_bigdata = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            'config_2dataset')

smalldata_winner = neat.nn.FeedForwardNetwork.create(data1_winner, config_smalldata)
bigdata_winner = neat.nn.FeedForwardNetwork.create(data2_winner, config_bigdata)

In [17]:
print(f"Количество узлов 1 датасета: {len(data1_winner.nodes)}")
print(f"Количество связей 1 датасета: {len(data1_winner.connections)}")
print(f"Количество узлов 2 датасета: {len(data2_winner.nodes)}")
print(f"Количество связей 2 датасета: {len(data2_winner.connections)}")

Количество узлов 1 датасета: 1
Количество связей 1 датасета: 8
Количество узлов 2 датасета: 3
Количество связей 2 датасета: 15


In [18]:
# Сохранение моделей
joblib.dump(data1_winner, '3_smalldata_winner.pkl')
joblib.dump(data2_winner, '3_bigdata_winner.pkl')

['3_bigdata_winner.pkl']